This Jupyter Notebook calculates the total loading of a field - the total mass of product applied to a field over a period of time. The result is displayed as a line chart in cell 6. User-defined parameters are located in cells 1 and 2. This code will display the total loading for one field whose ID is specified in cell 2, with the ID being the same identifier used in xCP to distinguish unique fields. It will also display a chart of total loading across all fields, as well as a chart of total loading symbolized by LULC type.

Version 2.0 - 11/6/2024 (Added charts for total loading for all fields and by LULC type)

Version 1.0 - 5/23/2024

Input file path

In [ ]:
# Output hdf file from xCropProtection
xcrop_arrdat_path = r'C:\path\to\arr.dat'

Select field/feature ID to chart

In [ ]:
feature_to_chart = 922889221

In [ ]:
import h5py
import datetime
import pandas
import geopandas
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Check that all subgroups are h5py datasets
def checkInstance(datasets):
    for dataset in datasets:
        if not isinstance(dataset, h5py.Dataset):
            return False
    return True

try:
    arr_file = h5py.File(xcrop_arrdat_path, 'r')
except FileNotFoundError:
    print("The file", xcrop_arrdat_path, "could not be accessed")

dataset = arr_file['xCropProtection']
landscape_dataset = arr_file['LandscapeScenario']

# Get data for subgroups
application_dates_subgroup = dataset['ApplicationDates']
application_rates_subgroup = dataset['ApplicationRates']
application_areas = dataset['AppliedAreas']
applied_features = dataset['AppliedFields']
application_PPP = dataset['AppliedPPP']
xcrop_file_path = dataset['xCropProtectionFilePath']
drift_reduction = dataset['TechnologyDriftReductions']

landscape_fields = landscape_dataset['FeatureIds']
landscape_feature_types = landscape_dataset['FeatureTypeIds']

# Check that subgroups are h5py datasets
if not checkInstance([application_dates_subgroup, application_rates_subgroup, application_PPP, xcrop_file_path, drift_reduction]):
    print("Error retrieving subgroup data.")
    quit

In [ ]:
application_dates_data = application_dates_subgroup[:]
application_rates_data = application_rates_subgroup[:]
applied_features_data = applied_features[:]
application_areas_data = application_areas[:]
feature_ids_data = landscape_fields[:]
feature_types_data = landscape_feature_types[:]

application_dates = [datetime.date.fromordinal(x) for x in application_dates_data]

feature_id_to_type_dict = {feature_ids_data[i]:feature_types_data[i] for i in range(0, len(feature_ids_data))}

# Convert application area arrays to bytes for geometry creation
app_areas_bytes = [x.tobytes() for x in application_areas] 

field_geometries = geopandas.GeoDataFrame(
    geometry=geopandas.GeoSeries.from_wkb(app_areas_bytes),
    crs="EPSG:3857"
).to_crs(crs="EPSG:4326")
field_geometries["field_idx"] = field_geometries.reset_index().index

# Project geometry and calculate area
geom_project = field_geometries.to_crs(crs="EPSG:3857")
geom_project_area_m = geom_project.area
geom_project_area_ha = geom_project_area_m / 10000

In [ ]:
# Calculate total loading for all fields and write to csv
field_info_set = {'featureID': applied_features_data, 'sumAppliedArea_ha': geom_project_area_ha, 'sumApplicationRate_g_per_ha': application_rates_data}
field_info_df = pandas.DataFrame(field_info_set)

# Show total loading for one field, specified by field_to_chart

# Create a DataFrame and set up columns
feature_information = pandas.DataFrame({'featureID': applied_features_data,
                                        'sumAppliedArea_ha': geom_project_area_ha,
                                        'sumApplicationRate_g_per_ha': application_rates_data,
                                        'applicationDate': application_dates})

fig, ax = plt.subplots()
# Plot the selected feature if it is in the dataframe
if feature_to_chart in applied_features_data:
    # Filter to only include indicated field and sort by date
    df_chart_feature_only = feature_information.loc[feature_information['featureID'] == feature_to_chart].sort_values(by=['applicationDate'])
    # Calculate total loading for each individual application
    feature_total_loading = (df_chart_feature_only.assign(totalLoading_g=df_chart_feature_only.eval('sumAppliedArea_ha * sumApplicationRate_g_per_ha')))
    
    # Calculate cumulative sum and chart
    cumulative_loading = feature_total_loading['totalLoading_g'].cumsum()
    plt.plot(feature_total_loading['applicationDate'], cumulative_loading)
    plt.title("Cumulative loading for field {}".format(feature_to_chart))
    plt.xlabel("Date")
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, horizontalalignment= 'right')
    plt.ylabel("Cumulative sum of mass applied (g)")
    plt.show()
else:
    # If the feature wasn't in the dataframe just plot the first feature
    print("Feature with ID", feature_to_chart, "is not present in the list of applied features.")
    print("Showing first feature in list. Feature ID:", applied_features_data[0])
    # Filter to only include indicated field and sort by date
    df_chart_feature_only = feature_information.loc[feature_information['featureID'] == applied_features_data[0]].sort_values(by=['applicationDate'])
    # Calculate total loading for each individual application
    feature_total_loading = (df_chart_feature_only.assign(totalLoading_g=df_chart_feature_only.eval('sumAppliedArea_ha * sumApplicationRate_g_per_ha')))
    
    # Calculate cumulative sum and chart
    cumulative_loading = feature_total_loading['totalLoading_g'].cumsum()
    plt.plot(feature_total_loading['applicationDate'], cumulative_loading)
    plt.title("Cumulative loading for field {}".format(applied_features_data[0]))
    plt.xlabel("Date")
    plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, horizontalalignment= 'right')
    plt.ylabel("Cumulative sum of mass applied (g)")
    plt.show()

Show the total loading for all fields

In [ ]:
# Sort by application date
df_chart_feature_only = feature_information.sort_values(by=['applicationDate'])
# Calculate total loading for each individual application
total_loading = (df_chart_feature_only.assign(totalLoading_g=df_chart_feature_only.eval('sumAppliedArea_ha * sumApplicationRate_g_per_ha')))

# Calculate cumulative sum and chart
cumulative_loading = total_loading['totalLoading_g'].cumsum()
fig, ax = plt.subplots()
plt.plot(total_loading['applicationDate'], cumulative_loading)
plt.title("Cumulative sum of mass applied to all fields")
plt.xlabel("Date")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, horizontalalignment= 'right')
plt.ylabel("Cumulative sum of mass applied (g)")
plt.show()

In [ ]:

feature_information = feature_information.assign(feature_LULC=lambda x: [feature_id_to_type_dict[feat_id] for feat_id in x['featureID'].values])

#Sort so the last date in the array will be the latest
application_dates.sort()

fig, ax = plt.subplots()
# Plot the selected feature if it is in the dataframe
for lulc in np.unique(feature_types_data):
    # Filter to only include lulc and sort by date
    df_chart_feature_only = feature_information.loc[feature_information['feature_LULC'] == lulc].sort_values(by=['applicationDate'])
    if len(df_chart_feature_only) == 0:
        continue
    # Calculate total loading for each individual application
    feature_total_loading = (df_chart_feature_only.assign(totalLoading_g=df_chart_feature_only.eval('sumAppliedArea_ha * sumApplicationRate_g_per_ha')))
    # Calculate cumulative sum and chart
    cumulative_loading = feature_total_loading['totalLoading_g'].cumsum().to_list()
    cumulative_loading.append(cumulative_loading[-1])
    date_list = feature_total_loading['applicationDate'].to_list()
    date_list.append(application_dates[-1])
    
    ax.plot(date_list, cumulative_loading, label=lulc)

plt.title("Cumulative sum of mass applied to all fields by LULC type")
plt.xlabel("Date")
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, horizontalalignment= 'right')
ax.set_ylabel('Cumulative sum of mass applied (g)')
fig.legend(loc='upper left', bbox_to_anchor=(.92, .92))
plt.show()